In [ ]:
from dldd import ClassificationModel
from pytorch_lightning import Trainer
from torch_geometric.data import DataLoader
from dldd.utils import TwoGraphData
import torch
import pickle

In [ ]:
with open('/srv/protected/final_list.pckl', 'rb') as file:
    data_list = pickle.load(file)

In [ ]:
def convert_to_class(data:TwoGraphData, threshold:int=6):
    data['label'] = torch.tensor(1, dtype=torch.long) if data['label'] < threshold else torch.tensor(0, dtype=torch.long)
    return data

In [ ]:
data_list = [TwoGraphData(**convert_to_class(x)) for x in data_list]

In [ ]:
# create DataLoader
train_dl = DataLoader(data_list,
                batch_size=32,
                num_workers=4,
                follow_batch=['protein_x', 'drug_x'])

In [ ]:
# create the model
model = ClassificationModel()

# train the model
trainer = Trainer(gpus=1, max_epochs=20)
trainer.fit(model, train_dataloaders=train_dl)